# Model training lab

This is the notebook for loading and training models.
Furthermore it provides simple documentation for different approaches used for training a model.

Run the command below to see command-completion on pressing `TAB`.

## Prerequisits

In [7]:
# Imports
import wandb
import os
import warnings
import tools
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.layers import SimpleRNN, Dense
from tensorflow.keras.layers import Bidirectional
from matplotlib import pyplot



# Ignore future warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Root CSV files directory
dirname = "./data/"  

# Constant frame count.
frames = 100


#Preparation Stage - Load data and normalize
listfile = os.listdir(dirname)
data = []
for wordname in listfile:
    if wordname == ".DS_Store":
        continue
    for csv in os.listdir(dirname + wordname):
        filepath = os.path.join(dirname, wordname, csv)
        content = pd.read_csv(filepath, sep=';')
        content = content.reindex(list(range(0, frames)), fill_value=0.0)
        content.fillna(0.0, inplace = True)
#        content = content.reindex(list(range(0, frames)), fill_value=-1.0)
#        content.fillna(-1.0, inplace = True) 
        data.append((wordname, content))
        
#Split data 60-20-20

features = [n[1] for n in data]
features = [f.to_numpy() for f in features]
labels = [n[0] for n in data]
x_train, x_val, y_train, y_val = train_test_split(features, labels, test_size=0.40, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_val, y_val, test_size=0.50, random_state=42)

#Enumerate
def printCountDataSets(dataset):
    wortCounter = []
    #Liste mit einmaligen Labels erstellen
    labels = sorted(set(dataset), key=dataset.index)
    #Liste nochmal Alphabetisch sortieren
    labels = sorted(labels)
    for label in labels:
        wortCounter.append(0)
    for row in dataset:
        for i in range(len(labels)):
            if str(labels[i]).startswith(row):
                wortCounter[i] += 1
    for i in range(len(labels)):
        print(labels[i], ': ', wortCounter[i], end =";  ")
    print(' ')        
    
print('Amount Datasets by word total:')
printCountDataSets(labels)
print('')

print('Amount Datasets by word training:')
printCountDataSets(y_train)
print('')

print('Amount Datasets by word validiation:')
printCountDataSets(y_val)
print('')

print('Amount Datasets by word test:')
printCountDataSets(y_test)
print('')


# Display data distribution
print('Distribution of data:')
print("Amount total:", len(labels))
print("Amount training:", len(y_train))
print("Amount validiation:", len(y_val))
print("Amount test:", len(y_test))
print('')

#Tokenize (One Hot)
tokenizer = tools.tokenize(dirname)
print('Tokens:')
print(tokenizer.word_index)
print('')
with open('tokens_json.txt', 'w') as outfile:
    outfile.write(tokenizer.to_json())

encoded_train=tokenizer.texts_to_sequences([y_train])[0]
encoded_val=tokenizer.texts_to_sequences([y_val])[0]
encoded_test=tokenizer.texts_to_sequences([y_test])[0]

y_train = to_categorical(encoded_train)
y_val = to_categorical(encoded_val)
y_test = to_categorical(encoded_test)

print('Categories in OneHot anotation:')
print(y_train)
print('')
# Making numpy arrays
x_train=np.array(x_train)
y_train=np.array(y_train)
x_val=np.array(x_val)
y_val=np.array(y_val)
x_test=np.array(x_test)
y_test=np.array(y_test)

print('Dataset coordinate Values:')
print(x_train)
print('')

import winsound
def finished(num):
    frequency = 2000  # Set Frequency To 2500 Hertz
    duration = 500  # Set Duration To 1000 ms == 1 second
    for i in range(0, num):
        winsound.Beep(frequency, duration)
    

Amount Datasets by word total:
Computer :  57;  Deutschland :  65;  Haben :  68;  Hallo :  57;  Mainz :  65;  Software :  67;  Welt :  66;  du :  66;  ich :  66;  unser :  64;  zeigen :  69;   

Amount Datasets by word training:
Computer :  31;  Deutschland :  30;  Haben :  38;  Hallo :  34;  Mainz :  38;  Software :  46;  Welt :  43;  du :  42;  ich :  37;  unser :  43;  zeigen :  44;   

Amount Datasets by word validiation:
Computer :  13;  Deutschland :  15;  Haben :  11;  Hallo :  11;  Mainz :  10;  Software :  13;  Welt :  14;  du :  13;  ich :  19;  unser :  10;  zeigen :  13;   

Amount Datasets by word test:
Computer :  13;  Deutschland :  20;  Haben :  19;  Hallo :  12;  Mainz :  17;  Software :  8;  Welt :  9;  du :  11;  ich :  10;  unser :  11;  zeigen :  12;   

Distribution of data:
Amount total: 710
Amount training: 426
Amount validiation: 142
Amount test: 142

Tokens:
{'computer': 1, 'deutschland': 2, 'du': 3, 'haben': 4, 'hallo': 5, 'ich': 6, 'mainz': 7, 'software': 8,

In [3]:
wandb.init(project="first_test_run_SLR")

Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable


W&B Run: https://app.wandb.ai/michid41/first_test_run_SLR/runs/37fb3nzl

## Training Stage
Configure the model and train it.

Metrics:
<div float="right">
    <img src="assets/accuracy.png" width="400"> 
    <img src="assets/precision_recall_formula.png" width="400">
</div>
<img src="assets/precision_recall.png" width="1000">


### <span style="color:blue"> Hyperparametertuned LSTM </span>
##### Here it is necessary to install the Keras-Tuner Module by executing:
#####  <span style="color:green"> via Conda:</span>
conda install -c conda-forge keras-tuner
#####  <span style="color:green"> for pip:</span>
pip install keras-tuner

Right now there are three different builds we are testing:
- classic LSTM
- CuDNNLSTM
- bidriectional LSTM

#####  <span style="color:red"> TODO: try to reduce learning rate or user tuner</span>
#####  <span style="color:red"> TODO: Crossfold</span>


In [8]:
from kerastuner.tuners import RandomSearch
from kerastuner.tuners import Hyperband
from kerastuner.engine.hyperparameters import HyperParameters
from time import time, strftime


starttime= strftime("%Y_%m_%d_%H%M%S")
LOG_DIR = "C:\ML\Optimization_"f"{starttime}" #<-In Windows below Log_dir Path will maybe be too long for Windows to handle, so use a shorter path like this here
#LOG_DIR = "./Optimization_"f"{starttime}" # LOG_DIR holds json files with information and a model of each single trial

def build_model_lstm(hp):
    model = Sequential()
#    model.add(tf.keras.layers.Masking(mask_value=-1.0, input_shape=(x_train.shape[1], x_train.shape[2])))
    model.add(layers.LSTM(hp.Int("LSTM_input", min_value =64, max_value=256,step=64, default=64), #kerastuner will randomly choose a value for nodes between 128 and 256 in steps of 64
                            return_sequences=True,
                            input_shape=(x_train.shape[1], x_train.shape[2])))
    
    for i in range(hp.Int("n_layers" , 1, 3)):    #number of layers ramdom between 1 an 3
        model.add(layers.LSTM(hp.Int(f"LSTM_{i}_units", min_value =64, max_value=256,step=64, default=64),return_sequences=True))
    
    model.add(layers.LSTM(hp.Int(f"LSTM_End", min_value =32, max_value=128,step=32, default=32)))
    model.add(layers.Dense(12, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
                  #optimizer=hp.Choice('optimizer',values=['Adam','RMSprop','SGD']),
                  optimizer=hp.Choice('optimizer',values=['Adagrad','Adamax']),
                  metrics=['accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])
    model.summary()
    print(model.optimizer.get_config()["name"])
    print('')
    return model



def build_model_CuDNNLSTM(hp):
    model = Sequential()
    

    
    model.add(tf.compat.v1.keras.layers.CuDNNLSTM(hp.Int("LSTM_input", min_value =64, max_value=256,step=64, default=64), #kerastuner will randomly choose a value for nodes between 128 and 256 in steps of 64
                            return_sequences=True,
                            input_shape=(x_train.shape[1], x_train.shape[2])))
    
    for i in range(hp.Int("n_layers" , 1, 3)):    #number of layers ramdom between 1 an 3
        model.add(tf.compat.v1.keras.layers.CuDNNLSTM(hp.Int(f"LSTM_{i}_units", min_value =64, max_value=256,step=64, default=64),return_sequences=True))
    
    model.add(tf.compat.v1.keras.layers.CuDNNLSTM(hp.Int(f"LSTM_End", min_value =32, max_value=128,step=32, default=32)))
    model.add(layers.Dense(12, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
                  #optimizer=hp.Choice('optimizer',values=['Adam','RMSprop','SGD']),
                  optimizer=hp.Choice('optimizer',values=['Adagrad','Adamax']),
                  metrics=['accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])
    model.summary()
    print(model.optimizer.get_config()["name"])
    print('')
    return model



def build_model_bdlstm(hp):
    model = Sequential()
    model.add(Bidirectional(layers.LSTM(hp.Int("LSTM_input", min_value =64, max_value=256,step=64, default=64),
                                        return_sequences=True),
                                        input_shape=(100, 86)))
    
    for i in range(hp.Int("n_layers" , 1, 3)):    #number of layers ramdom between 1 an 3
        model.add(layers.Bidirectional(layers.LSTM(hp.Int(f"LSTM_{i}_units", min_value =64, max_value=256,step=64, default=64),return_sequences=True)))
    
    model.add(layers.Bidirectional(layers.LSTM(hp.Int(f"LSTM_End", min_value =32, max_value=128,step=32, default=32))))
    model.add(layers.Dense(12, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
                  optimizer=hp.Choice('optimizer',values=['Adagrad','Adamax','Adam','RMSprop']),
                  metrics=['accuracy']) 
    model.summary()
    print(model.optimizer.get_config()["name"])
    print('')
    return model






###   <span style="color:red">Necesarry only in case of using Nvidia GPU  </span>

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU') 
print("Num GPUs:", len(physical_devices)) 

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

# Different Keras-Tuner Approaches
### 1 - RandomSearch
Parameter of variables are ranomly used (number of layers, number of nodes) and "best" model is chosen.

In [13]:
tuner  = RandomSearch(
    build_model_lstm,     #Function to use search in... See different builds above
    objective = "val_accuracy",  #Chooses "best model" looking for highest value of val_accuracy
    max_trials = 4,       # Number of different combinations tried Nodes and layers
    executions_per_trial = 3, 
    directory = LOG_DIR,
    project_name='SignLagnuageModelOptimization'
    )

#tuner.search_space_summary()

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, min_delta=0.1, patience=5, min_lr=0.001)

tuner.search(x=x_train,      #syntax just like in fit
                y= y_train,
            epochs=200,
            batch_size=32,
            validation_data=(x_val,y_val),
            verbose=2,
            callbacks=[reduce_lr]
            #callbacks=[wandb.keras.WandbCallback()]
            )

print(tuner.get_best_hyperparameters()[0].values)
print(tuner.results_summary())


# wandb.keras.WandbCallback()

finished(8)

INFO:tensorflow:Reloading Oracle from existing project C:\ML\Optimization_2020_07_15_201918\SignLagnuageModelOptimization\oracle.json
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 64)           38656     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 64)           33024     
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dense (Dense)                (None, 12)                396       
Total params: 84,492
Trainable params: 84,492
Non-trainable params: 0
_________________________________________________________________
Adagrad

INFO:tensorflow:Reloading Tuner from C:\ML\Optimization_2020_07_15_201918\SignLagnuageModelOptimization\tun

Epoch 33/200
426/426 - 1s - loss: 1.3850 - accuracy: 0.4624 - precision: 0.7719 - recall: 0.2066 - val_loss: 1.4748 - val_accuracy: 0.4014 - val_precision: 0.6600 - val_recall: 0.2324
Epoch 34/200
426/426 - 1s - loss: 1.3348 - accuracy: 0.4930 - precision: 0.7682 - recall: 0.2723 - val_loss: 1.4313 - val_accuracy: 0.4437 - val_precision: 0.7547 - val_recall: 0.2817
Epoch 35/200
426/426 - 1s - loss: 1.3577 - accuracy: 0.4718 - precision: 0.6614 - recall: 0.2934 - val_loss: 1.6190 - val_accuracy: 0.3662 - val_precision: 0.6667 - val_recall: 0.2254
Epoch 36/200
426/426 - 1s - loss: 1.4989 - accuracy: 0.4108 - precision: 0.6645 - recall: 0.2371 - val_loss: 1.4719 - val_accuracy: 0.4155 - val_precision: 0.6290 - val_recall: 0.2746
Epoch 37/200
426/426 - 1s - loss: 1.3575 - accuracy: 0.4648 - precision: 0.7564 - recall: 0.2770 - val_loss: 1.5446 - val_accuracy: 0.3944 - val_precision: 0.7447 - val_recall: 0.2465
Epoch 38/200
426/426 - 1s - loss: 1.4180 - accuracy: 0.4390 - precision: 0.7500 

Epoch 78/200
426/426 - 1s - loss: 1.0501 - accuracy: 0.6362 - precision: 0.7500 - recall: 0.4718 - val_loss: 1.1077 - val_accuracy: 0.5986 - val_precision: 0.7126 - val_recall: 0.4366
Epoch 79/200
426/426 - 1s - loss: 0.9230 - accuracy: 0.6690 - precision: 0.7810 - recall: 0.5023 - val_loss: 1.1327 - val_accuracy: 0.5563 - val_precision: 0.6782 - val_recall: 0.4155
Epoch 80/200
426/426 - 1s - loss: 0.8245 - accuracy: 0.6714 - precision: 0.8109 - recall: 0.5235 - val_loss: 0.9620 - val_accuracy: 0.5986 - val_precision: 0.7595 - val_recall: 0.4225
Epoch 81/200
426/426 - 1s - loss: 0.7405 - accuracy: 0.7042 - precision: 0.8316 - recall: 0.5563 - val_loss: 1.0711 - val_accuracy: 0.5775 - val_precision: 0.7619 - val_recall: 0.4507
Epoch 82/200
426/426 - 1s - loss: 0.8685 - accuracy: 0.6643 - precision: 0.8237 - recall: 0.5376 - val_loss: 1.1316 - val_accuracy: 0.5986 - val_precision: 0.7229 - val_recall: 0.4225
Epoch 83/200
426/426 - 1s - loss: 0.8674 - accuracy: 0.6784 - precision: 0.7957 

Epoch 123/200
426/426 - 1s - loss: 0.3440 - accuracy: 0.8732 - precision: 0.9128 - recall: 0.8357 - val_loss: 0.8317 - val_accuracy: 0.7113 - val_precision: 0.7405 - val_recall: 0.6831
Epoch 124/200
426/426 - 1s - loss: 0.3158 - accuracy: 0.9038 - precision: 0.9153 - recall: 0.8873 - val_loss: 0.8668 - val_accuracy: 0.7183 - val_precision: 0.7154 - val_recall: 0.6549
Epoch 125/200
426/426 - 1s - loss: 0.3054 - accuracy: 0.8920 - precision: 0.9193 - recall: 0.8826 - val_loss: 0.8689 - val_accuracy: 0.7113 - val_precision: 0.7176 - val_recall: 0.6620
Epoch 126/200
426/426 - 1s - loss: 0.3257 - accuracy: 0.8638 - precision: 0.8881 - recall: 0.8568 - val_loss: 0.8582 - val_accuracy: 0.6972 - val_precision: 0.7164 - val_recall: 0.6761
Epoch 127/200
426/426 - 1s - loss: 0.3595 - accuracy: 0.8803 - precision: 0.9132 - recall: 0.8638 - val_loss: 0.8353 - val_accuracy: 0.7254 - val_precision: 0.7557 - val_recall: 0.6972
Epoch 128/200
426/426 - 1s - loss: 0.3728 - accuracy: 0.8756 - precision: 0

Epoch 168/200
426/426 - 1s - loss: 0.1453 - accuracy: 0.9577 - precision: 0.9643 - recall: 0.9507 - val_loss: 0.9043 - val_accuracy: 0.7535 - val_precision: 0.7647 - val_recall: 0.7324
Epoch 169/200
426/426 - 1s - loss: 0.2212 - accuracy: 0.9131 - precision: 0.9211 - recall: 0.9038 - val_loss: 1.3387 - val_accuracy: 0.6549 - val_precision: 0.6544 - val_recall: 0.6268
Epoch 170/200
426/426 - 1s - loss: 0.3348 - accuracy: 0.8944 - precision: 0.9122 - recall: 0.8779 - val_loss: 1.0391 - val_accuracy: 0.6972 - val_precision: 0.7111 - val_recall: 0.6761
Epoch 171/200
426/426 - 1s - loss: 0.2918 - accuracy: 0.8991 - precision: 0.9069 - recall: 0.8920 - val_loss: 0.9187 - val_accuracy: 0.7676 - val_precision: 0.7754 - val_recall: 0.7535
Epoch 172/200
426/426 - 1s - loss: 0.2169 - accuracy: 0.9249 - precision: 0.9396 - recall: 0.9131 - val_loss: 0.9802 - val_accuracy: 0.7042 - val_precision: 0.7206 - val_recall: 0.6901
Epoch 173/200
426/426 - 1s - loss: 0.1757 - accuracy: 0.9390 - precision: 0

...

Epoch 75/200
426/426 - 1s - loss: 1.0781 - accuracy: 0.5681 - precision: 0.6642 - recall: 0.4178 - val_loss: 1.2133 - val_accuracy: 0.5704 - val_precision: 0.6196 - val_recall: 0.4014
Epoch 76/200
426/426 - 1s - loss: 1.0760 - accuracy: 0.5939 - precision: 0.6392 - recall: 0.4742 - val_loss: 1.3640 - val_accuracy: 0.4366 - val_precision: 0.5412 - val_recall: 0.3239
Epoch 77/200
426/426 - 1s - loss: 1.0554 - accuracy: 0.5845 - precision: 0.7055 - recall: 0.4554 - val_loss: 1.5196 - val_accuracy: 0.4155 - val_precision: 0.6197 - val_recall: 0.3099
Epoch 78/200
426/426 - 1s - loss: 0.9652 - accuracy: 0.6221 - precision: 0.7713 - recall: 0.4671 - val_loss: 1.2446 - val_accuracy: 0.5141 - val_precision: 0.6543 - val_recall: 0.3732
Epoch 79/200
426/426 - 1s - loss: 0.9752 - accuracy: 0.6315 - precision: 0.7361 - recall: 0.4977 - val_loss: 1.2215 - val_accuracy: 0.5493 - val_precision: 0.5865 - val_recall: 0.4296
Epoch 80/200
426/426 - 1s - loss: 1.0291 - accuracy: 0.6056 - precision: 0.7509 

Epoch 120/200
426/426 - 1s - loss: 0.5738 - accuracy: 0.7887 - precision: 0.8387 - recall: 0.7324 - val_loss: 1.2017 - val_accuracy: 0.6127 - val_precision: 0.6613 - val_recall: 0.5775
Epoch 121/200
426/426 - 1s - loss: 0.5659 - accuracy: 0.7934 - precision: 0.8385 - recall: 0.7559 - val_loss: 1.2371 - val_accuracy: 0.5986 - val_precision: 0.6696 - val_recall: 0.5423
Epoch 122/200
426/426 - 1s - loss: 0.5923 - accuracy: 0.7770 - precision: 0.8245 - recall: 0.7277 - val_loss: 1.1388 - val_accuracy: 0.6127 - val_precision: 0.6348 - val_recall: 0.5141
Epoch 123/200
426/426 - 1s - loss: 0.5768 - accuracy: 0.7817 - precision: 0.8436 - recall: 0.7089 - val_loss: 1.5625 - val_accuracy: 0.4859 - val_precision: 0.5167 - val_recall: 0.4366
Epoch 124/200
426/426 - 1s - loss: 0.5773 - accuracy: 0.7746 - precision: 0.8449 - recall: 0.7160 - val_loss: 1.2573 - val_accuracy: 0.5845 - val_precision: 0.6281 - val_recall: 0.5352
Epoch 125/200
426/426 - 1s - loss: 0.5684 - accuracy: 0.7723 - precision: 0

Epoch 165/200
426/426 - 1s - loss: 0.3891 - accuracy: 0.8826 - precision: 0.8960 - recall: 0.8498 - val_loss: 1.2476 - val_accuracy: 0.6268 - val_precision: 0.6772 - val_recall: 0.6056
Epoch 166/200
426/426 - 1s - loss: 0.3579 - accuracy: 0.8803 - precision: 0.8995 - recall: 0.8615 - val_loss: 1.4189 - val_accuracy: 0.5915 - val_precision: 0.6212 - val_recall: 0.5775
Epoch 167/200
426/426 - 1s - loss: 0.2906 - accuracy: 0.8920 - precision: 0.9193 - recall: 0.8826 - val_loss: 1.2937 - val_accuracy: 0.6197 - val_precision: 0.6641 - val_recall: 0.6127
Epoch 168/200
426/426 - 1s - loss: 0.3587 - accuracy: 0.8779 - precision: 0.8927 - recall: 0.8592 - val_loss: 1.3830 - val_accuracy: 0.6127 - val_precision: 0.6462 - val_recall: 0.5915
Epoch 169/200
426/426 - 1s - loss: 0.3371 - accuracy: 0.8756 - precision: 0.9015 - recall: 0.8592 - val_loss: 1.4272 - val_accuracy: 0.5986 - val_precision: 0.6183 - val_recall: 0.5704
Epoch 170/200
426/426 - 1s - loss: 0.3256 - accuracy: 0.8944 - precision: 0

INFO:tensorflow:Oracle triggered exit
{'LSTM_input': 192, 'n_layers': 3, 'LSTM_0_units': 192, 'LSTM_End': 96, 'optimizer': 'Adamax', 'LSTM_1_units': 128, 'LSTM_2_units': 64}


None


### 2 - Hyperband
Variation of RandomSearch http://jmlr.org/papers/volume18/16-558/16-558.pdf

In [ ]:
tuner  = Hyperband(
    build_model,
    objective = "val_accuracy",
    hyperband_iterations=2,
    max_epochs=150,
    directory = LOG_DIR,
    project_name='SignLagnuageModelOptimization'
    )

#tuner.search_space_summary()

tuner.search(x=x_train, 
            y= y_train,
            batch_size=32,
            validation_data=(x_val,y_val))

print(tuner.get_best_hyperparameters()[0].values)
print(tuner.results_summary())

finished(8)

In [ ]:
#Laut Randomsearch bestes Model am 23.06.2020
epochs=170
model = Sequential()
model.add(layers.LSTM(128, return_sequences=True,
               input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(layers.LSTM(64, return_sequences=True)) 
model.add(layers.LSTM(96))  
model.add(layers.Dense(12, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall(),tf.keras.metrics.AUC()])
model.summary()

history=model.fit(x_train,y_train,epochs=epochs,validation_data=(x_val,y_val),shuffle=False,verbose=2)

In [ ]:
prec=history.history['val_precision_2'][epochs-1]
rec=history.history['val_recall_2'][epochs-1]
print(prec)
print(rec)


f1= 2*((prec*rec)/(prec+rec))
print (f1)

In [ ]:
pyplot.plot(history.history['loss'])
pyplot.plot(history.history['val_loss'])
pyplot.title('model train vs validation loss')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'], loc='upper right')
pyplot.show()

pyplot.plot(history.history['accuracy'])
pyplot.plot(history.history['val_accuracy'])
pyplot.title('model train vs validation accuracy')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'], loc='upper right')
pyplot.show()

## Export tuner object into pickle file
so it can be used in other scripts

In [ ]:
import pickle

with open(f"tuner_"f"{starttime}.pkl", "wb") as f:
    pickle.dump(tuner, f)
    

## Get best Trial from Tuner Object

In [14]:
best_hp = tuner.get_best_hyperparameters()[0]
bestmodel= tuner.hypermodel.build(best_hp)

bestmodel.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 192)          214272    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 192)          295680    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100, 128)          164352    
_________________________________________________________________
lstm_3 (LSTM)                (None, 100, 64)           49408     
_________________________________________________________________
lstm_4 (LSTM)                (None, 96)                61824     
_________________________________________________________________
dense (Dense)                (None, 12)                1164      
Total params: 786,700
Trainable params: 786,700
Non-trainable params: 0
__________________________________________________

In [19]:
#tmp_chekpoints= "tmp\epoch{epoch:02d}-{val_accuracy:.2f}-{val_loss:.2f}.hdf5"
#tmp_chekpoints= "C:\\ML\\checkpoints\\tmp\\epoch{epoch:02d}-{val_accuracy:.2f}-{val_loss:.2f}.hdf5"
tmp_chekpoints= "C:\\ML\\checkpoints\\tmp\\epoch{epoch:02d}-{val_accuracy:.2f}-{val_loss:.2f}.hdf5"

#csv_log = tf.keras.callbacks.CSVLogger("log.csv", separator=',', append=False)
csv_log = tf.keras.callbacks.CSVLogger("C:\\ML\\logs\\log.csv", separator=',', append=False)

#tb = tf.keras.callbacks.TensorBoard(log_dir='logs', histogram_freq=1, write_graph=False, write_images=False, update_freq='epoch', profile_batch=2, embeddings_freq=1, embeddings_metadata=None)
tb = tf.keras.callbacks.TensorBoard(log_dir='C:\ML\logs', histogram_freq=1, write_graph=False, write_images=False, update_freq='epoch', profile_batch=2, embeddings_freq=1, embeddings_metadata=None)
es = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0.001, patience=20, verbose=0, mode='max', baseline=None, restore_best_weights=True)
chk= tf.keras.callbacks.ModelCheckpoint(tmp_chekpoints, monitor='val_accuracy', verbose=0, save_best_only=False, save_weights_only=False, mode='max', save_freq='epoch')
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, min_delta=0.1, patience=5, min_lr=0.001)

history = bestmodel.fit(x_train,y_train,epochs=200,batch_size=32, validation_data=(x_val,y_val),shuffle=False, verbose=2, callbacks=[csv_log, chk,reduce_lr])

Train on 426 samples, validate on 142 samples
Epoch 1/200
426/426 - 1s - loss: 2.1184 - accuracy: 0.2089 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 1.9701 - val_accuracy: 0.2746 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/200
426/426 - 1s - loss: 1.9338 - accuracy: 0.2629 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 1.8400 - val_accuracy: 0.2254 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/200
426/426 - 1s - loss: 1.8502 - accuracy: 0.2465 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 1.8005 - val_accuracy: 0.2254 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/200
426/426 - 1s - loss: 1.8488 - accuracy: 0.2840 - precision: 0.4667 - recall: 0.0164 - val_loss: 1.7787 - val_accuracy: 0.2324 - val_precision: 0.6250 - val_recall: 0.0352
Epoch 5/200
426/426 - 1s - loss: 1.8208 - accuracy: 0.2676 - precision: 0.5238 - recall: 0.0258 - val_loss: 1.7702 - val_accuracy: 0.2465 - val_precision: 1.0000 - val_recall

Epoch 45/200
426/426 - 1s - loss: 0.9083 - accuracy: 0.6620 - precision: 0.7601 - recall: 0.5282 - val_loss: 1.5464 - val_accuracy: 0.4296 - val_precision: 0.6173 - val_recall: 0.3521
Epoch 46/200
426/426 - 1s - loss: 1.3687 - accuracy: 0.4953 - precision: 0.6872 - recall: 0.3404 - val_loss: 1.2339 - val_accuracy: 0.5211 - val_precision: 0.7838 - val_recall: 0.4085
Epoch 47/200
426/426 - 1s - loss: 1.0951 - accuracy: 0.5610 - precision: 0.7425 - recall: 0.4061 - val_loss: 1.1194 - val_accuracy: 0.5563 - val_precision: 0.7011 - val_recall: 0.4296
Epoch 48/200
426/426 - 1s - loss: 0.9432 - accuracy: 0.6502 - precision: 0.7978 - recall: 0.5000 - val_loss: 1.1362 - val_accuracy: 0.5352 - val_precision: 0.7412 - val_recall: 0.4437
Epoch 49/200
426/426 - 1s - loss: 0.7883 - accuracy: 0.6995 - precision: 0.8112 - recall: 0.5446 - val_loss: 0.9849 - val_accuracy: 0.6127 - val_precision: 0.7684 - val_recall: 0.5141
Epoch 50/200
426/426 - 1s - loss: 0.7147 - accuracy: 0.7418 - precision: 0.8449 

Epoch 90/200
426/426 - 1s - loss: 0.5055 - accuracy: 0.7981 - precision: 0.8284 - recall: 0.7817 - val_loss: 0.8869 - val_accuracy: 0.6831 - val_precision: 0.7323 - val_recall: 0.6549
Epoch 91/200
426/426 - 1s - loss: 0.3760 - accuracy: 0.8685 - precision: 0.8936 - recall: 0.8474 - val_loss: 0.8648 - val_accuracy: 0.6901 - val_precision: 0.7252 - val_recall: 0.6690
Epoch 92/200
426/426 - 1s - loss: 0.3770 - accuracy: 0.8685 - precision: 0.8978 - recall: 0.8451 - val_loss: 0.9580 - val_accuracy: 0.6761 - val_precision: 0.6815 - val_recall: 0.6479
Epoch 93/200
426/426 - 1s - loss: 0.4648 - accuracy: 0.8357 - precision: 0.8564 - recall: 0.8122 - val_loss: 1.0151 - val_accuracy: 0.6620 - val_precision: 0.6889 - val_recall: 0.6549
Epoch 94/200
426/426 - 1s - loss: 0.4330 - accuracy: 0.8474 - precision: 0.8589 - recall: 0.8286 - val_loss: 0.8576 - val_accuracy: 0.7183 - val_precision: 0.7287 - val_recall: 0.6620
Epoch 95/200
426/426 - 1s - loss: 0.4142 - accuracy: 0.8568 - precision: 0.8806 

Epoch 135/200
426/426 - 1s - loss: 0.4687 - accuracy: 0.8239 - precision: 0.8390 - recall: 0.8075 - val_loss: 1.0380 - val_accuracy: 0.6901 - val_precision: 0.7029 - val_recall: 0.6831
Epoch 136/200
426/426 - 1s - loss: 0.3692 - accuracy: 0.8803 - precision: 0.8832 - recall: 0.8521 - val_loss: 0.9200 - val_accuracy: 0.6831 - val_precision: 0.7068 - val_recall: 0.6620
Epoch 137/200
426/426 - 1s - loss: 0.3423 - accuracy: 0.8779 - precision: 0.8846 - recall: 0.8638 - val_loss: 0.7519 - val_accuracy: 0.7324 - val_precision: 0.7481 - val_recall: 0.7113
Epoch 138/200
426/426 - 1s - loss: 0.2915 - accuracy: 0.8967 - precision: 0.9043 - recall: 0.8873 - val_loss: 0.7636 - val_accuracy: 0.7676 - val_precision: 0.7794 - val_recall: 0.7465
Epoch 139/200
426/426 - 1s - loss: 0.2058 - accuracy: 0.9366 - precision: 0.9405 - recall: 0.9272 - val_loss: 0.9155 - val_accuracy: 0.6831 - val_precision: 0.7176 - val_recall: 0.6620
Epoch 140/200
426/426 - 1s - loss: 0.3039 - accuracy: 0.8803 - precision: 0

Epoch 180/200
426/426 - 1s - loss: 0.2447 - accuracy: 0.9249 - precision: 0.9238 - recall: 0.9108 - val_loss: 1.0409 - val_accuracy: 0.7183 - val_precision: 0.7319 - val_recall: 0.7113
Epoch 181/200
426/426 - 1s - loss: 0.2418 - accuracy: 0.8991 - precision: 0.9233 - recall: 0.8756 - val_loss: 1.0065 - val_accuracy: 0.7183 - val_precision: 0.7481 - val_recall: 0.7113
Epoch 182/200
426/426 - 1s - loss: 0.1991 - accuracy: 0.9225 - precision: 0.9327 - recall: 0.9108 - val_loss: 0.9117 - val_accuracy: 0.7606 - val_precision: 0.7770 - val_recall: 0.7606
Epoch 183/200
426/426 - 1s - loss: 0.1075 - accuracy: 0.9577 - precision: 0.9668 - recall: 0.9577 - val_loss: 0.8826 - val_accuracy: 0.7676 - val_precision: 0.7794 - val_recall: 0.7465
Epoch 184/200
426/426 - 1s - loss: 0.1701 - accuracy: 0.9390 - precision: 0.9455 - recall: 0.9366 - val_loss: 0.8796 - val_accuracy: 0.7958 - val_precision: 0.8000 - val_recall: 0.7887
Epoch 185/200
426/426 - 1s - loss: 0.1589 - accuracy: 0.9460 - precision: 0

In [21]:
print(bestmodel.optimizer.get_config())


{'name': 'Adamax', 'learning_rate': 0.001, 'decay': 0.0, 'beta_1': 0.9, 'beta_2': 0.999, 'epsilon': 1e-07}


### Diagnostic Plots

The training history of your LSTM models can be used to diagnose the behavior of your model.

You can plot the performance of your model using the Matplotlib library. For example, you can plot training loss vs test loss as follows:

In [ ]:
pyplot.plot(history.history['loss'])
pyplot.plot(history.history['val_loss'])
pyplot.title('model train vs validation loss')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'], loc='upper right')
pyplot.savefig("C:/ML/loss"f"{starttime}.png")
pyplot.show()

pyplot.plot(history.history['accuracy'])
pyplot.plot(history.history['val_accuracy'])
pyplot.title('model train vs validation accuracy')
pyplot.ylabel('accuracy')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'], loc='lower right')
pyplot.savefig("C:/ML/accuracy_"f"{starttime}.png")
pyplot.show()

#### Underfit Example
Running this example produces a plot of train and validation loss showing the characteristic of an underfit model. In this case, performance may be improved by increasing the number of training epochs.


<img src="assets/Diagnostic-Line-Plot-Showing-an-Underfit-Model.png" width="400">


Running this example shows the characteristic of an underfit model that appears under-provisioned.
In this case, performance may be improved by increasing the capacity of the model, such as the number of memory cells in a hidden layer or number of hidden layers.

<img src="assets/Diagnostic-Line-Plot-Showing-an-Underfit-Model-via-Status.png" width="400">

#### Good Fit Example
Running the example creates a line plot showing the train and validation loss meeting.
Ideally, we would like to see model performance like this if possible, although this may not be possible on challenging problems with a lot of data.

<img src="assets/Diagnostic-Line-Plot-Showing-a-Good-Fit-for-a-Model.png" width="400">

#### Overfit Example
Running this example creates a plot showing the characteristic inflection point in validation loss of an overfit model.
This may be a sign of too many training epochs.
In this case, the model training could be stopped at the inflection point. Alternately, the number of training examples could be increased.

<img src="assets/Diagnostic-Line-Plot-Showing-an-Overfit-Model.png" width="400">

### Evaluate

In [ ]:
#model = tf.keras.models.load_model('./tmp/epoch49-0.90-0.39.hdf5')


#bestmodel.evaluate(x=x_test, y=y_test, verbose=2)
model.evaluate(x=x_test, y=y_test, verbose=2)


### Save model

In [ ]:
bestmodel.save("sign_lang_recognition_tuned.h5")